In [1]:
!wget https://data.cityofnewyork.us/api/views/hyij-8hr7/rows.csv?accessType=DOWNLOAD

--2016-11-09 18:11:40--  https://data.cityofnewyork.us/api/views/hyij-8hr7/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.205
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [        <=>           ] 185.12M  2.47MB/s   in 80s    

Last-modified header invalid -- time-stamp ignored.
2016-11-09 18:13:01 (2.31 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [194111037]



In [2]:
!mv rows.csv?accessType=DOWNLOAD crime.csv

In [3]:
!csvcut crime.csv | head -10 | csvlook

|-----------+------------+------------------------+-------------+------------------+----------------+-----------------+-----------------+----------------+--------------+---------------+--------------------------------+------------------------+--------+----------+-----------+---------------------+-------------+-------------+----------------------------------------|
|  OBJECTID | Identifier | Occurrence Date        | Day of Week | Occurrence Month | Occurrence Day | Occurrence Year | Occurrence Hour | CompStat Month | CompStat Day | CompStat Year | Offense                        | Offense Classification | Sector | Precinct | Borough   | Jurisdiction        | XCoordinate | YCoordinate | Location 1                             |
|-----------+------------+------------------------+-------------+------------------+----------------+-----------------+-----------------+----------------+--------------+---------------+--------------------------------+------------------------+--------+----------+---

In [4]:
!wc -l crime.csv

1123466 crime.csv


In [5]:
!csvcut crime.csv | csvgrep -c12 -m 'RAPE' > rape.csv

In [6]:
!csvcut -n rape.csv 

  1: OBJECTID
  2: Identifier
  3: Occurrence Date
  4: Day of Week
  5: Occurrence Month
  6: Occurrence Day
  7: Occurrence Year
  8: Occurrence Hour
  9: CompStat Month
 10: CompStat Day
 11: CompStat Year
 12: Offense
 13: Offense Classification
 14: Sector
 15: Precinct
 16: Borough
 17: Jurisdiction
 18: XCoordinate
 19: YCoordinate
 20: Location 1


In [7]:
!wc -l rape.csv

13780 rape.csv


In [27]:
!csvcut -c3,4,5,6,7,8,12,14,15,16,17,18,19 rape.csv > rape_final.csv

In [28]:
!csvcut rape_final.csv | head | csvlook

|-------------------------+-------------+------------------+----------------+-----------------+-----------------+---------+--------+----------+-----------+---------------------+-------------+--------------|
|  Occurrence Date        | Day of Week | Occurrence Month | Occurrence Day | Occurrence Year | Occurrence Hour | Offense | Sector | Precinct | Borough   | Jurisdiction        | XCoordinate | YCoordinate  |
|-------------------------+-------------+------------------+----------------+-----------------+-----------------+---------+--------+----------+-----------+---------------------+-------------+--------------|
|  08/08/1994 06:00:00 PM | Monday      | Aug              | 8              | 1994            | 18              | RAPE    | A      | 103      | QUEENS    | N.Y. POLICE DEPT    | 1041749     | 196938       |
|  02/01/1995 08:00:00 AM | Wednesday   | Feb              | 1              | 1995            | 8               | RAPE    | G      | 83       | BROOKLYN  | N.Y. POLICE DEPT

In [29]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [30]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [31]:
!createdb -U dbuser Project2

createdb: database creation failed: ERROR:  database "Project2" already exists


In [32]:
%sql postgresql://dbuser@localhost:5432/Project2

'Connected: dbuser@Project2'

In [33]:
!csvcut -n rape_final.csv

  1: Occurrence Date
  2: Day of Week
  3: Occurrence Month
  4: Occurrence Day
  5: Occurrence Year
  6: Occurrence Hour
  7: Offense
  8: Sector
  9: Precinct
 10: Borough
 11: Jurisdiction
 12: XCoordinate
 13: YCoordinate


In [34]:
!csvstat rape_final.csv

  1. Occurrence Date
	<class 'datetime.datetime'>
	Nulls: True
	Min: 1975-01-01 00:01:00
	Max: 2015-12-30 18:00:00
	Unique values: 12558
	5 most frequent values:
		2005-01-01 00:01:00:	23
		2007-01-01 00:01:00:	22
		2008-01-01 00:01:00:	20
		2006-01-01 00:01:00:	18
		2010-01-01 00:01:00:	18
  2. Day of Week
	<class 'str'>
	Nulls: True
	Unique values: 7
	5 most frequent values:
		Saturday:	2312
		Sunday:	2255
		Friday:	2067
		Monday:	1831
		Thursday:	1776
	Max length: 9
  3. Occurrence Month
	<class 'str'>
	Nulls: True
	Unique values: 12
	5 most frequent values:
		Jan:	1414
		Jul:	1299
		Aug:	1268
		Jun:	1266
		Oct:	1166
	Max length: 4
  4. Occurrence Day
	<class 'int'>
	Nulls: True
	Min: 1
	Max: 31
	Sum: 190088
	Mean: 13.79648715343301
	Median: 14.0
	Standard Deviation: 9.558368831423818
	Unique values: 31
	5 most frequent values:
		1:	2178
		15:	504
		20:	429
		10:	428
		25:	424
  5. Occurrence Year
	<class 'int'>
	Nulls: True
	Min: 1975
	Max: 2015
	Sum: 27688816
	Mean: 2009.931475029

In [40]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    offense CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )


Done.
Done.


[]

In [41]:
%%sql
COPY rapecrimes FROM '/home/jovyan/work/rape_final.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [42]:
%%sql
select * from rapecrimes
LIMIT 10;

10 rows affected.


date,day,month,day_n,year,hour,offense,sector,precint,borough,jur,xcor,ycor
1994-08-08 18:00:00,Monday,Aug,8,1994,18,RAPE,A,103,QUEENS,N.Y. POLICE DEPT,1041749,196938
1995-02-01 08:00:00,Wednesday,Feb,1,1995,8,RAPE,G,83,BROOKLYN,N.Y. POLICE DEPT,1006965,193571
2000-01-09 20:00:00,Sunday,Jan,9,2000,20,RAPE,G,108,QUEENS,N.Y. POLICE DEPT,996793,210001
2001-01-09 15:00:00,Tuesday,Jan,9,2001,15,RAPE,H,63,BROOKLYN,N.Y. HOUSING POLICE,1000448,168068
2001-02-11 00:01:00,Sunday,Feb,11,2001,0,RAPE,B,25,MANHATTAN,N.Y. POLICE DEPT,1000555,230994
2001-11-19 12:00:00,Monday,Nov,19,2001,12,RAPE,G,75,BROOKLYN,N.Y. POLICE DEPT,1017141,183798
2002-07-25 12:00:00,Thursday,Jul,25,2002,12,RAPE,A,101,QUEENS,N.Y. POLICE DEPT,1053678,159044
2002-07-30 12:00:00,Tuesday,Jul,30,2002,12,RAPE,E,47,BRONX,N.Y. POLICE DEPT,1026387,262634
2002-10-15 07:00:00,Tuesday,Oct,15,2002,7,RAPE,C,79,BROOKLYN,N.Y. POLICE DEPT,999634,190253
2002-11-25 00:01:00,Monday,Nov,25,2002,0,RAPE,B,17,MANHATTAN,N.Y. POLICE DEPT,992411,215025


In [43]:
%%sql
select year,count(*) from rapecrimes
GROUP BY year
LIMIT 10;

10 rows affected.


year,count
None,3
1975,1
1992,3
2008,1255
2004,115
1995,5
2014,1204
1994,6
1989,2
2012,1372


In [44]:
%%sql
select day,count(*) from rapecrimes
GROUP BY day


8 rows affected.


day,count
None,1
Thursday,1776
Wednesday,1769
Monday,1831
Tuesday,1768
Sunday,2255
Friday,2067
Saturday,2312


In [45]:
%%sql
select month,count(*) from rapecrimes
GROUP BY month

13 rows affected.


month,count
None,1
Mar,1060
Apr,1031
Oct,1166
Nov,1077
Jul,1299
Aug,1268
Sep,1159
May,1085
Jan,1414


# Problem 3

1.Now we need to create a table to store all data. 

In [46]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    offense CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )

Done.
Done.


[]

In [47]:
%%sql
COPY rapecrimes FROM '/home/jovyan/work/rape_final.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [48]:
%%sql
select * from rapecrimes
LIMIT 5;

5 rows affected.


date,day,month,day_n,year,hour,offense,sector,precint,borough,jur,xcor,ycor
1994-08-08 18:00:00,Monday,Aug,8,1994,18,RAPE,A,103,QUEENS,N.Y. POLICE DEPT,1041749,196938
1995-02-01 08:00:00,Wednesday,Feb,1,1995,8,RAPE,G,83,BROOKLYN,N.Y. POLICE DEPT,1006965,193571
2000-01-09 20:00:00,Sunday,Jan,9,2000,20,RAPE,G,108,QUEENS,N.Y. POLICE DEPT,996793,210001
2001-01-09 15:00:00,Tuesday,Jan,9,2001,15,RAPE,H,63,BROOKLYN,N.Y. HOUSING POLICE,1000448,168068
2001-02-11 00:01:00,Sunday,Feb,11,2001,0,RAPE,B,25,MANHATTAN,N.Y. POLICE DEPT,1000555,230994


Now create the table

In [49]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    offense CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )

Done.
Done.


[]

create the table of dimension time, and create the primary key

In [ ]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )

create the fact table of rape, and replace the time with foreigh key 

create the table of dimension location, and create the primary key

in the fact table of rape,replace the time with foreigh key 